In [4]:
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import RobustScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
from scipy.stats import pearsonr
from sklearn.svm import SVR  # 导入 SVR 模型

# **1. 读取数据**
df = pd.read_excel(r"C:\Users\Lyttl\Desktop\rice\ninanjie\data\combinedflower16.xlsx", header=None)

# **2. 处理数据格式**
df.columns = df.iloc[0]  # 第一行变成列名（样本名）
df = df.iloc[1:]  # 删除第一行
df.set_index(df.columns[0], inplace=True)  # 第一列 (基因 ID) 作为索引
df.index.name = "gene"

# **3. 确保所有数据是数值型**
df = df.apply(pd.to_numeric, errors="coerce")  # 转换为数值

# **4. 提取目标变量 (y) 和 特征矩阵 (X)**
y = df.loc["Flowering"].astype(float)  # Flowering 行作为目标变量
X = df.drop(index=["Flowering"]).T  # 删除 Flowering 行，并转置使样本为行，基因为列

print("转置前 X 形状:", df.drop(index=["Flowering"]).shape)
print("转置后 X 形状:", X.shape)
print("目标变量 y 形状:", y.shape)

# **5. 处理缺失值**
imputer = SimpleImputer(strategy="mean")
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns, index=X.index)

# **6. 进行标准化**
scaler = RobustScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_imputed), columns=X.columns, index=X.index)

# **7. 特征选择**
selector = SelectKBest(f_regression, k=10000)  # 选择前 10000 个最重要特征
X_selected = selector.fit_transform(X_scaled, y)

# 获取被选中的特征名称
selected_features = X_scaled.columns[selector.get_support()]

print("选择的特征数:", X_selected.shape[1])

# **8. 对目标变量 (y) 进行对数变换**
transformer = FunctionTransformer(np.log1p, validate=True)
y_transformed = transformer.fit_transform(y.values.reshape(-1, 1)).ravel()

# **9. 交叉验证 (KFold)**
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# **10. SVR 超参数优化**
param_grid_svr = {
    'C': [0.1, 1, 10],  # 惩罚系数
    'epsilon': [0.01, 0.1, 0.5],  # 误差敏感区
    'kernel': ['linear', 'rbf'],  # 核函数
    'gamma': ['scale', 'auto']  # 核函数系数
}

grid_search_svr = GridSearchCV(SVR(), param_grid_svr, cv=kf, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')
grid_search_svr.fit(X_selected, y_transformed)
best_svr_model = grid_search_svr.best_estimator_

# **11. 评估函数**
def evaluate(model, X, y_true):
    y_pred = model.predict(X)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    pcc, _ = pearsonr(y_true, y_pred)
    return mse, r2, mae, rmse, pcc

# **12. 训练 & 评估**
def train_and_evaluate(model, X_train, X_val, X_test, y_train, y_val, y_test):
    model.fit(X_train, y_train)
    mse_train, r2_train, mae_train, rmse_train, pcc_train = evaluate(model, X_train, y_train)
    mse_val, r2_val, mae_val, rmse_val, pcc_val = evaluate(model, X_val, y_val)
    mse_test, r2_test, mae_test, rmse_test, pcc_test = evaluate(model, X_test, y_test)
    
    return r2_train, r2_val, r2_test, mse_train, mse_val, mse_test, mae_train, mae_val, mae_test, rmse_train, rmse_val, rmse_test, pcc_train, pcc_val, pcc_test

# **13. 数据集划分**
X_train, X_temp, y_train, y_temp = train_test_split(X_selected, y_transformed, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# **14. 评估模型**
metrics_svr = train_and_evaluate(best_svr_model, X_train, X_val, X_test, y_train, y_val, y_test)

# **15. 输出评估结果**
def print_metrics(name, metrics):
    print(f"{name} - 训练集 R²: {metrics[0]}, 验证集 R²: {metrics[1]}, 测试集 R²: {metrics[2]}")
    print(f"{name} - 训练集 MSE: {metrics[3]}, 验证集 MSE: {metrics[4]}, 测试集 MSE: {metrics[5]}")
    print(f"{name} - 训练集 PCC: {metrics[12]}, 验证集 PCC: {metrics[13]}, 测试集 PCC: {metrics[14]}")

print_metrics("SVR with Tuning", metrics_svr)

# **16. SHAP 可解释性分析**
explainer = shap.KernelExplainer(best_svr_model.predict, X_train[:50])  # 取部分数据作为背景值
shap_values = explainer.shap_values(X_train[:200])  # 计算 SHAP 值（避免计算量过大）

# 转换 SHAP 值为 NumPy 数组
shap_values = np.array(shap_values)

# 创建特征名称对应的 DataFrame
X_selected_df = pd.DataFrame(X_selected, columns=selected_features)

# 生成 SHAP Summary Plot（特征重要性）
shap.summary_plot(shap_values, X_selected_df.iloc[:200], max_display=10)

# 生成 SHAP 条形图
shap.summary_plot(shap_values, X_selected_df.iloc[:200], plot_type="bar", max_display=10)


转置前 X 形状: (24175, 383)
转置后 X 形状: (383, 24175)
目标变量 y 形状: (383,)
选择的特征数: 10000
Fitting 5 folds for each of 36 candidates, totalling 180 fits


C:\Users\Lyttl\Desktop\nky_rice\.venv\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


SVR with Tuning - 训练集 R²: 0.7418426398691651, 验证集 R²: 0.21987365423425276, 测试集 R²: 0.14254773928674846
SVR with Tuning - 训练集 MSE: 0.022571421787664723, 验证集 MSE: 0.0717060279447448, 测试集 MSE: 0.08316861969159849
SVR with Tuning - 训练集 PCC: 0.8909828259502832, 验证集 PCC: 0.48587176637611, 测试集 PCC: 0.42904754538910167


  0%|          | 0/200 [00:37<?, ?it/s]


MemoryError: Unable to allocate 82.1 GiB for an array with shape (22048, 500000) and data type float64

In [5]:
# **16. SHAP 可解释性分析**
# 选择较小的训练集数据计算 SHAP 值
X_train_small = X_train[:50]  # 只使用前 50 个样本作为背景值
explainer = shap.KernelExplainer(best_svr_model.predict, X_train_small)  # 使用较小的背景数据
shap_values = explainer.shap_values(X_train[:100])  # 计算前 100 个样本的 SHAP 值

# 转换 SHAP 值为 NumPy 数组
shap_values = np.array(shap_values)

# 创建特征名称对应的 DataFrame
X_selected_df = pd.DataFrame(X_selected, columns=selected_features)

# 生成 SHAP Summary Plot（特征重要性）
shap.summary_plot(shap_values, X_selected_df.iloc[:100], max_display=10)

# 生成 SHAP 条形图
shap.summary_plot(shap_values, X_selected_df.iloc[:100], plot_type="bar", max_display=10)


  0%|          | 0/100 [00:10<?, ?it/s]


MemoryError: Unable to allocate 82.1 GiB for an array with shape (22048, 500000) and data type float64